# Create sampling points for Rhino and Modelica

Stupid random sampling. More systematic ways: explore pyDOE! 
- https://pythonhosted.org/pyDOE/index.html
- https://github.com/clicumu/pyDOE2

How to identify things

- Project (projectid: Kiss1)
- Study: a variant within the parameters of the project (studyid -> vergeben vom Optimierer oder wie hier)
- Geometry Model (parametric) -> github commit id nutzen
- Geometry Instance -> id von rhino vergeben
- Analysis model (parametric) -> github commit id nutzen
- analysis instance -> von analyse vergeben

geometrie ist auch eine analyse!!

Welche Rolle spielt CAD? nur am ende zur Doku...

### Input parameters (for Rhino)

- number of racks
- number of cells per rack
- cooling channel width
- cooling channel height
- wall thickness

### Input parameters for OpenModelica

- cover (wall between cell and pipe) 
  - area_h = 3.44e-4, 
  - c_wall = 888, 
  - k_wall = 220,
  - rho_wall = 2700, 
  - s = 0.002
- "wall" between two cells
  - area_h = 1.72e-4, 
  - c_wall = 888, 
  - k_wall = 220, 
  - rho_wall = 2700, 
  - s = 0.04
- pipe
  - crossArea = 0.32e-4, 
  - length = 0.043, 
  - perimeter = 0.024, 
- mass flow source
  - m_flow = 0.0037,
- Gc_Thermischer_Kontaktwiderstand(k = 3.311) 



In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from uuid import uuid1
import json
import random
import os


In [4]:
# constants
constants = {
    "C_quartercell": 36.3,
    "k_cell": 3.311, 

    "c_cover": 888.,
    "k_cover": 220.,
    "rho_cover": 2700.,

    "c_plate": 888.,
    "k_plate": 220.,
    "rho_plate": 2700.,

    "channel_dist": 0.04,

    "cell_width": 0.043,
    "stoptime": 2000,
}

# Configurations for number of racks and number of cells. 
# total number of cells in a battery pack is 2*nracks*ncells
# varies among 140 and 144
# key: nracks, value: ncells
cellconfigs = {4: 18, 5: 14, 6: 12, 7: 10, 8: 9,
               9: 8, 10: 7, 12: 6, 14: 5, 18: 4, 
               #3: 24, 24: 3, 
               #2: 35, 35: 2
              }

In [5]:
# number of points to sample
N = 200

# baseline as first dataset
df = pd.DataFrame(data={
    "t_cover": [0.002] + [random.uniform(0.5e-3, 2.0e-3) for _ in range(N)],
    "channel_width": [0.008] + [random.uniform(4e-3, 19e-3) for _ in range(N)], 
    "channel_height": [0.004] + [random.uniform(0.5e-3, 4e-3) for _ in range(N)],
    "nracks": [9] + random.choices(list(cellconfigs.keys()), k=N)
})

# compute ncells
df["ncells"] = df.nracks.apply(lambda nr: cellconfigs[nr])

# add constants
for k, v in constants.items():
    df[k] = v
    
# constant m_flow (per rack)
df['m_flow'] = 120/3600/df['nracks'] # * [random.uniform(0.1, 10) for _ in range(N)]
    
# drop rows with w < h
#df.query("channel_width >= channel_height", inplace=True)
    
# compute modelica stuff
df["area_cover"] = df["channel_width"] * df["cell_width"]
df["t_plate"] = df["channel_dist"] - df["channel_width"]
df["area_plate"] = df["channel_height"] * df["cell_width"] #  + 2 * t_cover??
df["area_channel"] = df["channel_height"] * df["channel_width"]
df["perim_channel"] = 2 * (df["channel_height"] + df["channel_width"])
df["ar_channel"] = df["channel_width"] / df["channel_height"]
df["fluid_name"] = "Novec7000"

# create simulation id
df["simid"] = [str(uuid1()) for _ in range(len(df))]

In [6]:
df.head()

t_cover  channel_width  channel_height  nracks  ncells  C_quartercell  \
0  0.002000       0.008000        0.004000       9       8           36.3   
1  0.000974       0.013062        0.003171      12       6           36.3   
2  0.001375       0.012859        0.000502       9       8           36.3   
3  0.001612       0.005368        0.003704       8       9           36.3   
4  0.001865       0.007566        0.001206      18       4           36.3   

   k_cell  c_cover  k_cover  rho_cover  ...  stoptime    m_flow  area_cover  \
0   3.311    888.0    220.0     2700.0  ...      2000  0.003704    0.000344   
1   3.311    888.0    220.0     2700.0  ...      2000  0.002778    0.000562   
2   3.311    888.0    220.0     2700.0  ...      2000  0.003704    0.000553   
3   3.311    888.0    220.0     2700.0  ...      2000  0.004167    0.000231   
4   3.311    888.0    220.0     2700.0  ...      2000  0.001852    0.000325   

    t_plate  area_plate  area_channel  perim_channel  ar_channel  fluid_name  \
0  0.032000    0.000172      0.000032       0.024000    2.000000   Novec7000   
1  0.026938    0.000136      0.000041       0.032467    4.119425   Novec7000   
2  0.027141    0.000022      0.000006       0.026722   25.614512   Novec7000   
3  0.034632    0.000159      0.000020       0.018145    1.449128   Novec7000   
4  0.032434    0.000052      0.000009       0.017544    6.272863   Novec7000   

                                  simid  
0  121c6903-9eaa-11eb-b6c1-7470fd3cb1c0  
1  121c6904-9eaa-11eb-b3d4-7470fd3cb1c0  
2  121c6905-9eaa-11eb-9490-7470fd3cb1c0  
3  121c6906-9eaa-11eb-8d58-7470fd3cb1c0  
4  121c6907-9eaa-11eb-b7be-7470fd3cb1c0  

[5 rows x 25 columns]

In [7]:
# save as csv
df.to_csv("200_samples_novec.csv", index=False)

## Input for Rhino
https://github.com/slzefzal/KISS-Battery/blob/main/input1.json

```
{
    "data": 
    [
        {
        "assy_id": "Battery_01",
        "nracks": 3,
        "ncells": 5,
        "channel_width": 20.0,
        "channel_height": 1.5,
        "t_wall": 1.2,
        "t_baseplate_without_channel": 2.0,
        "t_endplate": 2.0
        }
    ]
}
```

- "assy_id" ... simid
- "t_wall" ... t_cover
- "t_baseplate_without_channel": 2.0 (constant)
- "t_endplate": 2.0 (constant)

In [9]:
# save as individual json files for Rhino
rhino_dir = "inp_rhino"
#df = pd.read_csv("200_samples_Vdot_const.csv")

def to_rhino(din, t_baseplate_without_channel=2.0, t_endplate=2.0):
    """Return a dictionary in the form above"""
    simid = din["simid"]
    data = dict(assy_id=simid, 
                nracks=din["nracks"], 
                ncells=din["ncells"],
                channel_width=din["channel_width"],
                channel_height=din["channel_height"],
                t_wall=din["t_cover"],
                t_baseplate_without_channel=t_baseplate_without_channel,
                t_endplate=t_endplate
               )
    return {"data": [data]}

for din in df.to_dict(orient='index').values():
    with open(os.path.join(rhino_dir, "%s.json"%din["simid"]), "w", encoding="utf-8") as fp:
        json.dump(to_rhino(din), fp, indent=2)

In [10]:
# write a shell script for computations. 
# in DOS: use the START command to run jobs in background
bindir = "C:\\Users\\Oliver\\Projects\\battery_pack_v2\\data_novec\\bin"

n_jobs = 8
cols = ['t_cover', 'm_flow', 'area_cover', 't_plate', 
        'area_plate', 'area_channel', 'perim_channel']

with open(os.path.join(bindir, "runall.bat"), "w") as fp:
    for i, row in enumerate(df.itertuples()):
        params = ",".join([f"{c}={getattr(row, c)}" for c in cols])
        command = f"Kiss{row.nracks}x{row.ncells}.exe -override {params} -r ..\\res\\{row.simid}_res.mat\n"
        if (i+1)%n_jobs == 0:
            fp.write(f"{command}")
        else:
            fp.write(f"start /B {command}")
        

In [13]:
1e5*0.02**2/12

3.3333333333333335

In [15]:
.5*.1*20**2/.5**2

80.0